In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
from urllib.request import urlopen
from tqdm import tqdm_notebook
import numpy as np

In [2]:
df_debut_artists = pd.read_csv("../data/debut_album_1118.csv")

In [3]:
df_debut_artists.head()

,artist,album,single_count
0,2Cellos,2Cellos,3
1,Deep Dark Robot,8 Songs About a Girl,0
2,The Young Professionals,"9am to 5pm, 5pm to Whenever\n(09:00 to 17:00, ...",0
3,Flash Bang Grenada,10 Haters,0
4,Her Majesty & the Wolves,111,3


In [319]:
list(df_artists['artist'][-7:])

['Lil Xan',
 'YoungBoy Never Broke Again',
 'EDEN',
 'Nipsey Hussle',
 'Rich the Kid',
 'Myles Kennedy',
 'Fickle Friends']

In [51]:
df_debut_artists[df_debut_artists['artist'].isin(['Rich Brian'])]

,artist,album,single_count
1906,Rich Brian,Amen,3


In [52]:
artist = df_debut_artists['artist'][1906]
album = df_debut_artists['album'][1906]

In [53]:
artist_ = artist.replace(" ", "+")
album_ = album.replace(" ", "+")

In [54]:
url_ = 'https://genius.com/api/search/article?page=' + str(1) + '&q=' + artist_ + "+" + album_
url_

'https://genius.com/api/search/article?page=1&q=Rich+Brian+Amen'

In [55]:
_response = requests.get(url_).json()
_response

{'meta': {'status': 200},
 'response': {'next_page': 2,
  'sections': [{'hits': [{'highlights': [],
      'index': 'article',
      'result': {'_type': 'article',
       'api_path': '/articles/4247',
       'article_type': 'news',
       'author': {'_type': 'user',
        'about_me_summary': 'Staff Writer at Genius. Brooklyn native and resident cat lady.',
        'api_path': '/users/5506789',
        'avatar': {'medium': {'bounding_box': {'height': 400, 'width': 300},
          'url': 'https://images.genius.com/avatars/medium/f4a7e20636cbe9b0456cae0600b9f40c'},
         'small': {'bounding_box': {'height': 100, 'width': 100},
          'url': 'https://images.genius.com/avatars/small/f4a7e20636cbe9b0456cae0600b9f40c'},
         'thumb': {'bounding_box': {'height': 32, 'width': 32},
          'url': 'https://images.genius.com/avatars/thumb/f4a7e20636cbe9b0456cae0600b9f40c'},
         'tiny': {'bounding_box': {'height': 16, 'width': 16},
          'url': 'https://images.genius.com/avata

In [332]:
def get_genius(data):
    
    df_articles = pd.DataFrame(columns = ['artist', 'album', 'genius_frequency'])
    
    def submitform(name):
        result = name.replace(" ", "+")
        return result
    
    for i in tqdm_notebook(range(len(list(data['artist'])))):

        artist = df_debut_artists['artist'][i]
        album = df_debut_artists['album'][i]
        
        artist_ = artist.replace(" ", "+")
        album_ = album.replace(" ", "+")
        
        page_num = 1
        articles = 0

        while True:

            url_ = 'https://genius.com/api/search/article?page=' + str(page_num) + '&q=' + artist_ + "+" + album_
            response = requests.get(url_).json()

            articles += len(response['response']['sections'][0]['hits'])

            page_num += 1

            if response['response']['next_page'] == None :
                break

        data = {
            'artist': artist,
            'album' : album,
            'genius_frequency' : articles
        }

        print(data)

        df_articles.loc[len(df_articles)] = data
    
    return df_articles

In [334]:
genius_articles = get_genius(df_debut_artists)

HBox(children=(IntProgress(value=0, max=1949), HTML(value='')))

Exception in thread Thread-26:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [336]:
print(genius_articles.shape)
genius_articles.tail()

(1949, 2)


,artist,genius_frequency
1944,EDEN,33
1945,Nipsey Hussle,6
1946,Rich the Kid,17
1947,Myles Kennedy,0
1948,Fickle Friends,0


In [338]:
genius_articles.to_csv("../data/buzz_genius.csv", index=False)